In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.linear_model import LinearRegression
%matplotlib inline
from statsmodels.formula.api import ols
import numpy as np
import src
import random
import scipy.stats as stats
df1 = pd.read_csv("kc_house_data_train.csv")
df = pd.read_csv("kc_house_data_test_features.csv")

In [11]:
def remove_outliers(x):
    std = x.std()
    two_std = x.mean() + 3*std
    return np.where(x < two_std, x, two_std)

In [12]:
df['sqft_living_n'] = remove_outliers(df.sqft_living)
df['bedrooms_n'] = remove_outliers(df.bedrooms)
df['sqft_above_n'] = remove_outliers(df.sqft_above)

In [13]:
lat_long = df1.groupby(['lat', 'long'])['price'].mean().sort_values(ascending = False)[0:10].index

for x in range(0,len(lat_long)):
    df['ll'+str(x)] = ((df.lat - lat_long[x][0])**2 + (df.long - lat_long[x][1])**2) 
df.head()

,Unnamed: 0,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,ll0,ll1,ll2,ll3,ll4,ll5,ll6,ll7,ll8,ll9
0,0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,...,0.012981,0.004198,0.006148,0.006464,0.007785,0.022219,0.001937,0.024035,0.003153,0.008246
1,1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,...,0.012981,0.004198,0.006148,0.006464,0.007785,0.022219,0.001937,0.024035,0.003153,0.008246
2,2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,...,0.112448,0.057224,0.065503,0.061900,0.055060,0.045197,0.062139,0.045040,0.120368,0.054258
3,3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,...,0.076250,0.029042,0.041150,0.039194,0.036481,0.051577,0.023956,0.053805,0.043725,0.036764
4,4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,...,0.054081,0.032274,0.030794,0.028984,0.025142,0.010798,0.041293,0.009898,0.088769,0.024331


In [14]:
df = pd.get_dummies(df, columns = ['zipcode'], drop_first = True)
zipcodes = []
for x in df.columns:
    if 'zipcode' in x:
        zipcodes.append(x)

In [15]:
df['sq_sqft'] = df['sqft_living'] **2


In [16]:
df['has_been_renovated'] = df['yr_renovated'].apply(lambda x: x>0).astype(int)

In [17]:
df['ren_in_lastyr'] = np.where(df['yr_renovated'] >=2014, 1, 0)

In [18]:
df = pd.get_dummies(df, columns = ['grade'], drop_first = True)
df['grade_3'] = 0

In [19]:
df['month'] = df['date'].apply(lambda x: x[4:6])

In [20]:
df.head()

,Unnamed: 0,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade_6,grade_7,grade_8,grade_9,grade_10,grade_11,grade_12,grade_13,grade_3,month
0,0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,...,0,0,1,0,0,0,0,0,0,08
1,1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,...,0,0,1,0,0,0,0,0,0,02
2,2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,...,0,0,1,0,0,0,0,0,0,11
3,3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,...,0,0,1,0,0,0,0,0,0,12
4,4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,...,0,0,1,0,0,0,0,0,0,01


In [21]:
df['month'] = df['month'].astype(int)
df['fall'] = df['month'].apply(lambda x: x>9).astype(int)
df['winter'] = df['month'].apply(lambda x: x<3).astype(int)
df['spring'] = df['month'].apply(lambda x: 6>x>3).astype(int)
df['summer'] = df['month'].apply(lambda x: 9>x>6).astype(int)
df.head(20)

,Unnamed: 0,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade_10,grade_11,grade_12,grade_13,grade_3,month,fall,winter,spring,summer
0,0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,...,0,0,0,0,0,8,0,0,0,1
1,1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,...,0,0,0,0,0,2,0,1,0,0
2,2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,...,0,0,0,0,0,11,1,0,0,0
3,3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,...,0,0,0,0,0,12,1,0,0,0
4,4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,...,0,0,0,0,0,1,0,1,0,0
5,5,1870400605,20141117T000000,4,2.25,1970,7125,1.5,0,0,...,0,0,0,0,0,11,1,0,0,0
6,6,4047200825,20141011T000000,1,1.00,1390,60984,1.0,0,0,...,0,0,0,0,0,10,1,0,0,0
7,7,7304300430,20141201T000000,4,1.00,1020,11383,1.5,0,0,...,0,0,0,0,0,12,1,0,0,0
8,8,8079010190,20140626T000000,4,2.50,2250,7526,2.0,0,0,...,0,0,0,0,0,6,0,0,0,0
9,9,1453600182,20140528T000000,2,1.00,800,6240,1.0,0,0,...,0,0,0,0,0,5,0,0,1,0


In [23]:
df = pd.get_dummies(df, columns = ['view'], drop_first = True)


In [24]:
df['bathrooms_1'] = (df['bathrooms'] ==1).astype(int)
df['bathrooms_2'] = (df['bathrooms'] ==2).astype(int)
df['bathrooms_3'] = (df['bathrooms'] ==3).astype(int)
df['bathrooms_4'] = (df['bathrooms'] ==4).astype(int)
df['bathrooms_5'] = (df['bathrooms'] ==5).astype(int)
df.head()

,Unnamed: 0,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,condition,...,summer,view_1,view_2,view_3,view_4,bathrooms_1,bathrooms_2,bathrooms_3,bathrooms_4,bathrooms_5
0,0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,3,...,1,0,0,0,0,0,0,0,0,0
1,1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,3,...,0,0,0,0,0,0,0,0,0,0
2,2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,3,...,0,0,0,0,0,0,0,0,0,0
3,3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,3,...,0,0,0,0,0,0,0,0,0,0
4,4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,3,...,0,0,0,0,0,0,0,0,0,0


In [297]:
df.head()

,Unnamed: 0,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,...,price_log,view_1,view_2,view_3,view_4,bathrooms_1,bathrooms_2,bathrooms_3,bathrooms_4,bathrooms_5
0,0,2591820310,20141006T000000,365000.0,4,2.25,2070,8893,2.0,0,...,12.807653,0,0,0,0,0,0,0,0,0
1,7,7010701383,20141017T000000,680000.0,3,2.50,1800,4400,1.0,0,...,13.429848,0,0,0,0,0,0,0,0,0
2,17,2386000070,20141029T000000,795127.0,4,3.25,4360,91158,1.0,0,...,13.586257,0,0,0,0,0,0,0,0,0
3,18,1138000450,20141016T000000,355000.0,4,1.00,1440,7215,1.5,0,...,12.779873,0,0,0,0,1,0,0,0,0
4,23,9324800450,20141009T000000,560000.0,3,1.50,2790,6900,1.0,0,...,13.235692,0,1,0,0,0,0,0,0,0


# RMSE SCORE

This cell is just to get my factors into a useable list for the sklearn model. 

In [42]:
factors =                """zipcode_98004+zipcode_98005+
                            zipcode_98006+zipcode_98007+zipcode_98008+zipcode_98010 
                            +zipcode_98011+zipcode_98014+zipcode_98019 
                            +zipcode_98024+zipcode_98027+zipcode_98028 
                            +zipcode_98029+
                            zipcode_98033+zipcode_98034+zipcode_98039 
                            +zipcode_98040+zipcode_98045+zipcode_98052 
                            +zipcode_98053+zipcode_98056
                            +zipcode_98059+zipcode_98065+zipcode_98072 
                            +zipcode_98074+zipcode_98075+zipcode_98077+zipcode_98092 
                            +zipcode_98102+zipcode_98103+zipcode_98105 
                            +zipcode_98107+zipcode_98108+zipcode_98109+zipcode_98112 
                            +zipcode_98115+zipcode_98116+zipcode_98117+zipcode_98118 
                            +zipcode_98119+zipcode_98122+zipcode_98125+zipcode_98126 
                            +zipcode_98133+zipcode_98136+zipcode_98144+zipcode_98146 
                            +zipcode_98155+zipcode_98166 
                            +zipcode_98177+zipcode_98199+sq_sqft+sqft_living
                            +grade_3+view_1+view_2+view_3+view_4
                            +grade_4+grade_5+grade_6+grade_7+grade_8
                            +grade_9+grade_10+grade_11+grade_12+waterfront
                            +sqft_living15+winter+fall+sqft_basement+bathrooms_1+bathrooms_2
                            +bedrooms
                            +ll1+ll0+ll2+ll3+ll4+ll5+ll6+ll7+ll8+ll9"""
factors = factors.replace(' ', '')
factors = factors.replace('\n', '')
factors = factors.split('+')
df[factors]

,zipcode_98004,zipcode_98005,zipcode_98006,zipcode_98007,zipcode_98008,zipcode_98010,zipcode_98011,zipcode_98014,zipcode_98019,zipcode_98024,...,ll1,ll0,ll2,ll3,ll4,ll5,ll6,ll7,ll8,ll9
0,0,0,0,0,0,0,0,0,0,0,...,0.004198,0.012981,0.006148,0.006464,0.007785,0.022219,0.001937,0.024035,0.003153,0.008246
1,0,0,0,0,0,0,0,0,0,0,...,0.004198,0.012981,0.006148,0.006464,0.007785,0.022219,0.001937,0.024035,0.003153,0.008246
2,0,0,0,0,0,0,0,0,0,0,...,0.057224,0.112448,0.065503,0.061900,0.055060,0.045197,0.062139,0.045040,0.120368,0.054258
3,0,0,0,0,0,0,0,0,0,0,...,0.029042,0.076250,0.041150,0.039194,0.036481,0.051577,0.023956,0.053805,0.043725,0.036764
4,0,0,0,0,0,0,0,0,0,0,...,0.032274,0.054081,0.030794,0.028984,0.025142,0.010798,0.041293,0.009898,0.088769,0.024331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4318,0,0,0,0,0,0,0,0,0,0,...,0.019854,0.005359,0.015969,0.017725,0.021667,0.037046,0.018736,0.038745,0.006856,0.022291
4319,0,0,0,0,0,0,0,0,0,0,...,0.041308,0.015706,0.029236,0.030612,0.032820,0.025850,0.050357,0.025248,0.063654,0.032571
4320,0,0,0,0,0,0,0,0,0,0,...,0.010316,0.001829,0.004784,0.005546,0.007070,0.008901,0.014517,0.009320,0.024355,0.007097
4321,0,0,0,0,0,0,0,0,0,0,...,0.034365,0.073598,0.038457,0.035807,0.030669,0.020699,0.040385,0.020387,0.090660,0.029948


In [43]:
import pickle
model = open("model.pickle", "rb")
model = pickle.load(model)

In [44]:
predictions = model.predict(df[factors])

In [45]:
predictions = np.exp(predictions)
predictions_df = pd.Series(predictions)

In [49]:
predictions_df.to_csv('predictions2.csv', index = False)

/Users/aidancoco/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
